In [1]:
import numpy as np
import pandas as pd
from pyecharts import Pie

In [2]:
train_df = pd.read_table("./Input/train_20171215.txt",dtype='int')

In [3]:
test_df = pd.read_table("./Input/test_A_20171225.txt",dtype='int')

In [4]:
from pyecharts import Pie

attr = ["训练数据","测试数据"]
v1 = [train_df.shape[0],test_df.shape[0]]
pie = Pie("训练集数据集比例","训练数据->train_df, 测试数据->test_df")
pie.use_theme("dark")
pie.add("", attr, v1, is_label_show=True)
pie

题目提供三年数据，预测一年数据

In [5]:
from pyecharts import Scatter

v1 = train_df["date"].values
v2 = train_df["cnt"].values
scatter = Scatter("训练集上牌量","三年数据时间相关，有一定周期性")
scatter.use_theme("dark")
scatter.add("cnt", v1, v2)
scatter

In [6]:
from pyecharts import Boxplot

boxplot = Boxplot()
x_axis = ['trian']
y_axis = [train_df["cnt"]]
boxplot.use_theme("dark")
_yaxis = boxplot.prepare_data(y_axis)
boxplot.add("boxplot", x_axis, _yaxis)
boxplot

In [7]:
a = train_df["cnt"].value_counts(bins=30).sort_index().rolling(20)
a

Rolling [window=20,center=False,axis=0]

In [8]:
from pyecharts import Bar, Line, Overlap
train_cnt = train_df["cnt"].value_counts().sort_index()
train_cnt_norm = train_df["cnt"].value_counts(normalize=True).sort_index()
v1 = train_cnt.values
v2 = train_cnt_norm.values
v2 = pd.rolling_mean(v2,30)
attr1 = train_cnt.index
attr2 = train_cnt_norm.index
bar = Bar()
bar.use_theme("dark")
bar.add("cnt", attr1, v1, mark_point=["average"])
line = Line()
line.use_theme("dark")
line.add("cnt_norm", attr2, v2, mark_point=["max"],is_smooth=True)
overlap = Overlap()
overlap.add(bar)
overlap.use_theme("dark")
overlap.add(line, yaxis_index=1, is_add_yaxis=True)
overlap
# TODO：bins，norm

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: pd.rolling_mean is deprecated for ndarrays and will be removed in a future version
  


In [9]:
print(train_df['cnt'].describe())

count    4773.000000
mean      380.567358
std       252.720918
min        12.000000
25%       221.000000
50%       351.000000
75%       496.000000
max      2102.000000
Name: cnt, dtype: float64


In [10]:
train_df.head()

,date,day_of_week,brand,cnt
0,1,3,1,20
1,1,3,5,48
2,2,4,1,16
3,2,4,3,20
4,3,5,1,1411


In [11]:
from pyecharts import Bar, Grid
grid = Grid(width=900,height=850)
grid.use_theme("dark")
top = 30
left = 70
row = 1

for week in set(train_df["day_of_week"]):
    tmp_df = train_df[train_df["day_of_week"]==week][["date","cnt"]]
    bar = Bar()
    bar.add(str(week), tmp_df["date"], tmp_df["cnt"], 
            mark_line=["min","max","average"], legend_top=top, legend_pos=left)
    grid.add(bar,grid_top=top, grid_left=left, grid_width=300, grid_height=150)
    row = (row+1)%2
    top = top + 200*row
    left = (left + 450) % 900

grid


按照星期合并数据

In [12]:
train_by_date_df = train_df.drop("brand",axis = 1).groupby(by='date',as_index=False).first()
train_by_date_df["cnt"] = train_df.drop("brand",axis = 1).groupby(by='date',as_index=False).sum()["cnt"]

In [13]:
from pyecharts import Scatter
v1 = train_by_date_df['date'].index
v2 = train_by_date_df['cnt'].values
scatter = Scatter("日期分组统计上牌量", "有明显的大小日分布，周末和平日的特征明显，峰值存在周期性")
scatter.use_theme("dark")
scatter.add("cnt_group_by_date", v1, v2, is_visualmap=True)
scatter

In [14]:
date_datet = pd.read_csv("./Input/train_data_0225_corect_su.csv")[["date","date_t"]]

In [15]:
date_datet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1507 entries, 0 to 1506
Data columns (total 2 columns):
date      1507 non-null float64
date_t    1507 non-null object
dtypes: float64(1), object(1)
memory usage: 23.6+ KB


In [16]:
train_df = train_df.merge(date_datet,on="date")

In [17]:
train_df.head()

,date,day_of_week,brand,cnt,date_t
0,1,3,1,20,2013-01-02
1,1,3,5,48,2013-01-02
2,2,4,1,16,2013-01-03
3,2,4,3,20,2013-01-03
4,3,5,1,1411,2013-01-04


In [18]:
train_df["date_t"] = train_df["date_t"].map(lambda x:str.replace(x,'-','/'))

In [19]:
data = []
for train in train_df[:50].sort_values("date_t").values:
    data.append([str(train[4]),train[3],str(train[2])])

In [20]:
from pyecharts import ThemeRiver
tr = ThemeRiver("最初两周销量")
tr.use_theme("dark")
tr.add(['1', '2', '3', '4', '5'], data, is_label_show=True)
tr

In [21]:
train_df
from pyecharts import Scatter
v1 = train_by_date_df['day_of_week'].values
v2 = train_by_date_df['cnt'].values
scatter = Scatter("", "")
scatter.use_theme("dark")
scatter.add("cnt", v1, v2, is_visualmap=True)
scatter